In [32]:
import urllib.request
import datetime
import time
import json
from config import *
import math
import requests
from bs4 import BeautifulSoup
import folium
import pandas as pd

In [3]:
def get_request_url(url):
    
    req = urllib.request.Request(url)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [4]:
#[CODE 1]

def SanggaInfoService(pageNo,numOfRows,sigunguCd,bjdongCd):
    
    end_point = "http://api.suwon.go.kr/openapi-data/service/SanggaInfo/getHjdsearchInfo"
    
    parameters = "?pageNo=" + str(pageNo)
    parameters += "&numOfRows=" + str(numOfRows)
    parameters += "&sigunguCd=" + str(sigunguCd)
    parameters += "&bjdongCd=" + str(bjdongCd)
    parameters += "&ServiceKey=" + "TIyuV2FHFQSS46wIlsJNrwYGs9SqQEDOtNb0XZjVWp8k6pVUitI9BsXYzNGEYhUmLuovIzoeoIEA3fbXwqQwTg%3D%3D"
    
    global url
    
    url = end_point + parameters
    
    global retData
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return

In [5]:
sigunguCd_suwon = [41111,41113,41115,41117]
sigungu_suwon = ['장안구','권선구','팔달구','영통구']

In [7]:
cafe = []

for i in sigunguCd_suwon :
    
    SanggaInfoService(1,5000,i,00000)

    soup = BeautifulSoup(retData, "html.parser")
    items = soup.find_all("item")

    for j in items :
        if str(j.indsmclsnm.text) == "커피점/카페" :
            cafe.append(j)

In [8]:
list(set(cafe))
len(cafe)

516

In [33]:
jangan = []
kwonsun = []
paldal = []
yeongtong = []

for i in cafe:
    if ('장안구' in i.rdnmadr.text.split()) == True :
        jangan.append(i)
    elif ('권선구' in i.rdnmadr.text.split()) == True :
        kwonsun.append(i)
    elif ('팔달구' in i.rdnmadr.text.split()) == True :
        paldal.append(i)
    else :
        yeongtong.append(i)

In [34]:
cafe_series = pd.Series([len(jangan),len(kwonsun),len(paldal),len(yeongtong)],
                        index = sigungu_suwon)
cafe_series

장안구    159
권선구     87
팔달구    145
영통구    125
dtype: int64

In [36]:
cafe_series.sum(axis=0)

516

In [20]:
geo_path = '수원시_geo.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [42]:
map_cafe = folium.Map([36.472531, 127.991828], zoom_start = 12)
map_cafe.choropleth(geo_data = geo_str, data = cafe_series,
               columns = [cafe_series.index, cafe_series],
               key_on = 'feature.id',
               fill_color = 'PuRd',
               legend_name = '구별 카페 갯수')

for i in cafe :
    if ('스타벅스' in i.bizesnm.text) == True :
        folium.CircleMarker([float(i.lat.text),float(i.lon.text)], radius = 5,
                            popup = i.bizesnm.text, color = 'green',
                            fill_color = '#5000cc').add_to(map_cafe)
        
    if ('이디야' in i.bizesnm.text) == True :
        folium.CircleMarker([float(i.lat.text),float(i.lon.text)], radius = 5,
                            popup = i.bizesnm.text, color = 'blue',
                            fill_color = '#5000cc').add_to(map_cafe)
map_cafe